In [1]:
# LendingClub Data Statistical Analysis
# Two sources of data and a data dictionary
# Data in CSV and SQL files are provided - Kaggle
# Looks like FICO score data is missing and some column names are different
import math
import sqlite3
import csv
import pandas as pd
import collections
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dateutil.parser import parse
from datetime import datetime
from statsmodels.stats import weightstats as stests
from scipy.stats import ks_2samp

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

dataURL = 'https://www.kaggle.com/wendykan/lending-club-loan-data/downloads/lending-club-loan-data.zip'
homeDir = "/fdata/LendingClub/lending-club-loan-data/"
fnSQL = "database.sqlite"
fnCSV = "loan.csv"

# list of sub grade categories
sg = ['A1','A2','A3','A4','A5',\
      'B1','B2','B3','B4','B5',\
      'C1','C2','C3','C4','C5',\
      'D1','D2','D3','D4','D5',\
      'E1','E3','E3','E4','E5',\
      'F1','F2','F3','F4','F5',\
      'G1','G2','G3','G4','G5']

# read in column names from loan.csv
with open(homeDir+fnCSV) as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    cnCSV = next(reader)[0].split(',')
#print(len(cnCSV))
#print(cnCSV)
# 74 Data Columns

# get column of data from CSV file
def getCSVColumn(header):
    try:
        indx = cnCSV.index(header)
        with open(homeDir+fnCSV) as csvfile:
            reader = csv.reader(csvfile)
            tcol = list(zip(*reader))[indx]
            col = list(tcol)
            col.pop(0)
            return col
    except ValueError:
        print("Column not found:", header)
        col = []
        return col

# get column of data from SQLite file
def getSQLiteColumn(header):
    col = []
    conn = sqlite3.connect(homeDir+fnSQL)
    cursor = conn.execute('SELECT ' + header + ' FROM loan')
    for row in cursor.fetchall():
        col.append(row[0])
    return col

# statistical KS comparison of factor across sub grades
# test distributions are the same of factor in columns 10 sub-grade (2 grades) apart
def slidingKSTest(subgrade, factor, space):
    p = np.zeros(shape=(35 - space))
    for i in range(35 - space):
        s1 = sg[i]
        s2 = sg[i+space]
        t1 = []
        t2 = []
        for indx, entry in enumerate(subgrade):
            if entry == s1:
                if not math.isnan(factor[indx]):
                    t1.append(factor[indx])
            if entry == s2:
                if not math.isnan(factor[indx]):
                    t2.append(factor[indx])
        #print(len(t1),len(t2))
        a, b = ks_2samp(t1,t2)
        p[i] = b
    return p

# Statistical KS comparison between A grade and factor from all other sub-grades
# test 'distributions are the same'
def slidingAGrade_KSTest(subgrade, factor):
    p = np.zeros(shape=(30))
    for i in range(30):
        s1 = 'A'
        s2 = sg[i+5]
        t1 = []
        t2 = []
        for indx, entry in enumerate(subgrade):
            if entry[0:1] == s1:
                if not math.isnan(factor[indx]):
                    t1.append(factor[indx])
            if entry == s2:
                if not math.isnan(factor[indx]):
                    t2.append(factor[indx])
        #print(len(t1),len(t2))
        a, b = ks_2samp(t1,t2)
        p[i] = b
    return p

# Compare default/charge off to paid off
def compareGB(subgrade, status, factor,fname):
    fg = []
    sgg = []
    fb = []
    sgb = []
    for indx, entry in enumerate(status):
        if (entry == 'Fully Paid'):
            if not math.isnan(factor[indx]):
                fg.append(factor[indx])
                sgg.append(subgrade[indx]+'G')
        elif ((entry == 'Charged Off') | (entry == 'Default')):
            if not math.isnan(factor[indx]):
                fb.append(factor[indx])
                sgb.append(subgrade[indx]+'B')         
    gdf = pd.DataFrame(list(zip(sgg,fg)),columns=['Sub Grade',fname])
    bdf = pd.DataFrame(list(zip(sgb,fb)),columns=['Sub Grade',fname])
    return gdf, bdf

# Compare Bad/Good Loans using KS test
# each subgrade
# each total population
def GBKSTest(subgrade, status,factor,fname):
    good, bad = compareGB(subgrade,status,factor,fname)
    KSs = []
    for indx, entry in enumerate(sg):
        gn = good.loc[good['Sub Grade'] == entry+'G'][fname].tolist()
        bn =  bad.loc[bad['Sub Grade'] == entry+'B'][fname].tolist()
        a, b = ks_2samp(gn,bn)
        KSs.append(b)
    gna = good[fname].tolist()
    bna = bad[fname].tolist()
    a, KSa = ks_2samp(gna,bna)
    return KSs, KSa

In [87]:
# load columns used for fitting from sql file and compare to ones from csv source
grablist_sql = ['id','loan_status','dti','revol_bal','acc_now_delinq','collections_12_mths_ex_med','delinq_2yrs',\
            'emp_length','inq_last_6mths','inq_fi','il_util','earliest_cr_line','annual_inc','revol_util',\
            'open_acc','tot_coll_amt','verification_status']
dcheck_sql = pd.DataFrame()
for entry in grablist_sql:
    temp = pd.DataFrame(getSQLiteColumn(entry))
    dcheck_sql = pd.concat([dcheck_sql, temp], axis=1)
dcheck_sql.columns = grablist_sql

In [88]:
grablist_csv = ['id','loan_status','dti','revol_bal','acc_now_delinq','collections_12_mths_ex_med','delinq_2yrs',\
            'emp_length','inq_last_6mths','inq_fi','il_util','earliest_cr_line','annual_inc','revol_util',\
            'open_acc','tot_coll_amt','verification_status']
dcheck_csv = pd.DataFrame()
for entry in grablist_csv:
    temp = pd.DataFrame(getCSVColumn(entry))
    dcheck_csv = pd.concat([dcheck_csv, temp], axis=1)
dcheck_csv.columns = grablist_sql

In [89]:
#pickle them
dcheck_sql.to_pickle(homeDir+'DCheckSQL.pkl')
dcheck_csv.to_pickle(homeDir+'DCheckCSV.pkl')

In [125]:
dcheck_sql = pd.read_pickle(homeDir+'DCheckSQL.pkl')
dcheck_csv = pd.read_pickle(homeDir+'DCheckCSV.pkl')

In [126]:
# do a bunch of data processing for both data frames
from dateutil.parser import parse
from datetime import datetime
dcheck_csv = dcheck_csv.replace(r'', np.nan, regex=True)
# Income Verification to numeric code
dcheck_csv.loc[dcheck_csv['verification_status'] == 'Not Verified', 'IVCode'] = 0
dcheck_csv.loc[dcheck_csv['verification_status'] == 'Source Verified', 'IVCode'] = 1
dcheck_csv.loc[dcheck_csv['verification_status'] == 'Verified', 'IVCode'] = 2
dcheck_sql.loc[dcheck_sql['verification_status'] == 'Not Verified', 'IVCode'] = 0
dcheck_sql.loc[dcheck_sql['verification_status'] == 'Source Verified', 'IVCode'] = 1
dcheck_sql.loc[dcheck_sql['verification_status'] == 'Verified', 'IVCode'] = 2
# Loan Status to numeric code
# Only use default, charged off (bad) and fully paid (good)
dcheck_csv.loc[(dcheck_csv['loan_status'] == 'Fully Paid') | \
          (dcheck_csv['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid'),'LSCode'] = 0
dcheck_csv.loc[(dcheck_csv['loan_status'] == 'Charged Off') | (dcheck_csv['loan_status'] == 'Default') | \
          (dcheck_csv['loan_status'] == 'Does not meet the credit policy. Status:Charged Off'),'LSCode'] = 1
dcheck_sql.loc[(dcheck_sql['loan_status'] == 'Fully Paid') | \
          (dcheck_sql['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid'),'LSCode'] = 0
dcheck_sql.loc[(dcheck_sql['loan_status'] == 'Charged Off') | (dcheck_sql['loan_status'] == 'Default') | \
          (dcheck_sql['loan_status'] == 'Does not meet the credit policy. Status:Charged Off'),'LSCode'] = 1
# convert years employed to code
dcheck_csv.loc[dcheck_csv['emp_length'] == '10+ years','ELCode'] = 10.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '< 1 year','ELCode'] = 0.5
dcheck_csv.loc[dcheck_csv['emp_length'] == '3 years','ELCode'] = 3.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '9 years','ELCode'] = 9.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '4 years','ELCode'] = 4.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '5 years','ELCode'] = 5.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '1 year','ELCode'] = 1.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '6 years','ELCode'] = 6.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '2 years','ELCode'] = 2.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '7 years','ELCode'] = 7.0
dcheck_csv.loc[dcheck_csv['emp_length'] == '8 years','ELCode'] = 8.0
dcheck_csv.loc[dcheck_csv['emp_length'] == 'n/a','ELCode'] = np.nan
dcheck_sql.loc[dcheck_sql['emp_length'] == '10+ years','ELCode'] = 10.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '< 1 year','ELCode'] = 0.5
dcheck_sql.loc[dcheck_sql['emp_length'] == '3 years','ELCode'] = 3.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '9 years','ELCode'] = 9.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '4 years','ELCode'] = 4.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '5 years','ELCode'] = 5.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '1 year','ELCode'] = 1.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '6 years','ELCode'] = 6.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '2 years','ELCode'] = 2.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '7 years','ELCode'] = 7.0
dcheck_sql.loc[dcheck_sql['emp_length'] == '8 years','ELCode'] = 8.0
dcheck_sql.loc[dcheck_sql['emp_length'] == 'n/a','ELCode'] = np.nan
#convert earliest credit to number of days
ec_csv = dcheck_csv['earliest_cr_line'].tolist()
for indx, element in enumerate(ec_csv):
    if (element != '') & (isinstance(element,str)):
        ec_csv[indx] = (parse('Jan-2016') - parse(element)).days
    else:
        ec_csv[indx] = np.nan
        
ec_sql = dcheck_sql['earliest_cr_line'].tolist()
for indx, element in enumerate(ec_sql):
    if (element != '') & (isinstance(element,str)):
        ec_sql[indx] = (parse('Jan-2016') - parse(element)).days
    else:
        ec_sql[indx] = np.nan
        
dcheck_csv['ECDays'] = pd.DataFrame(ec_csv,columns=['ECDays'])
dcheck_sql['ECDays'] = pd.DataFrame(ec_sql,columns=['ECDays'])
del dcheck_csv['earliest_cr_line']
del dcheck_sql['earliest_cr_line']
del dcheck_csv['emp_length']
del dcheck_sql['emp_length']
del dcheck_sql['loan_status']
del dcheck_csv['loan_status']
del dcheck_sql['verification_status']
del dcheck_csv['verification_status']


In [127]:
tnum = ['id','dti','revol_bal','acc_now_delinq','collections_12_mths_ex_med','delinq_2yrs','inq_last_6mths',\
       'inq_fi','il_util','annual_inc','open_acc','tot_coll_amt']
dcheck_csv[tnum] = dcheck_csv[tnum].apply(pd.to_numeric,errors='coerce')
dcheck_sql['id'] = dcheck_sql['id'].apply(pd.to_numeric,errors='coerce')
dcheck_csv['revol_util'] = dcheck_csv['revol_util'].apply(pd.to_numeric,errors='coerce')
dcheck_sql['revol_util'] = dcheck_sql['revol_util'].map(lambda x: str(x)[:-1])
dcheck_sql['revol_util'] = dcheck_sql['revol_util'].apply(pd.to_numeric,errors='coerce')
print(dcheck_csv.dtypes)
print(dcheck_sql.dtypes)

id                              int64
dti                           float64
revol_bal                     float64
acc_now_delinq                float64
collections_12_mths_ex_med    float64
delinq_2yrs                   float64
inq_last_6mths                float64
inq_fi                        float64
il_util                       float64
annual_inc                    float64
revol_util                    float64
open_acc                      float64
tot_coll_amt                  float64
IVCode                        float64
LSCode                        float64
ELCode                        float64
ECDays                        float64
dtype: object
id                            float64
dti                           float64
revol_bal                     float64
acc_now_delinq                float64
collections_12_mths_ex_med    float64
delinq_2yrs                   float64
inq_last_6mths                float64
inq_fi                        float64
il_util                       float6

In [128]:
#pickle them
dcheck_sql.to_pickle(homeDir+'DCheckSQL_F.pkl')
dcheck_csv.to_pickle(homeDir+'DCheckCSV_F.pkl')

In [129]:
dcheck_sql = pd.read_pickle(homeDir+'DCheckSQL_F.pkl')
dcheck_csv = pd.read_pickle(homeDir+'DCheckCSV_F.pkl')
del dcheck_sql['inq_fi']
del dcheck_sql['il_util']
del dcheck_sql['tot_coll_amt']
del dcheck_csv['inq_fi']
del dcheck_csv['il_util']
del dcheck_csv['tot_coll_amt']
print(dcheck_csv.head())
dcheck_sql.dropna(inplace=True)
dcheck_csv.dropna(inplace=True)
print(dcheck_sql.count())
print(dcheck_csv.count())

        id    dti  revol_bal  acc_now_delinq  collections_12_mths_ex_med  \
0  1077501  27.65    13648.0             0.0                         0.0   
1  1077430   1.00     1687.0             0.0                         0.0   
2  1077175   8.72     2956.0             0.0                         0.0   
3  1076863  20.00     5598.0             0.0                         0.0   
4  1075358  17.94    27783.0             0.0                         0.0   

   delinq_2yrs  inq_last_6mths  annual_inc  revol_util  open_acc  IVCode  \
0          0.0             1.0     24000.0        83.7       3.0     2.0   
1          0.0             5.0     30000.0         9.4       3.0     1.0   
2          0.0             2.0     12252.0        98.5       2.0     0.0   
3          0.0             1.0     49200.0        21.0      10.0     1.0   
4          0.0             0.0     80000.0        53.9      15.0     1.0   

   LSCode  ELCode   ECDays  
0     0.0    10.0  11322.0  
1     1.0     0.5   6119.0  

In [130]:
print(dcheck_csv.head())
print(dcheck_sql.head())

        id    dti  revol_bal  acc_now_delinq  collections_12_mths_ex_med  \
0  1077501  27.65    13648.0             0.0                         0.0   
1  1077430   1.00     1687.0             0.0                         0.0   
2  1077175   8.72     2956.0             0.0                         0.0   
3  1076863  20.00     5598.0             0.0                         0.0   
5  1075269  11.20     7963.0             0.0                         0.0   

   delinq_2yrs  inq_last_6mths  annual_inc  revol_util  open_acc  IVCode  \
0          0.0             1.0     24000.0        83.7       3.0     2.0   
1          0.0             5.0     30000.0         9.4       3.0     1.0   
2          0.0             2.0     12252.0        98.5       2.0     0.0   
3          0.0             1.0     49200.0        21.0      10.0     1.0   
5          0.0             3.0     36000.0        28.3       9.0     1.0   

   LSCode  ELCode   ECDays  
0     0.0    10.0  11322.0  
1     1.0     0.5   6119.0  

In [132]:
#pickle them
dcheck_sql.to_pickle(homeDir+'DCheckSQL_FF.pkl')
dcheck_csv.to_pickle(homeDir+'DCheckCSV_FF.pkl')